In [1]:
import sys
sys.path.insert(0, '../nsga-net/')
from search import train_search
import os
import pandas as pd
import plotly.graph_objs as go
import hashlib
import plotly.io as pio
import ast
nas_log_dir = '/home/nima/genologs/search-GA-BiObj-micro-20240425-060906'
import json
from datetime import datetime

/home/nima/miniconda3/envs/bittensor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nima/miniconda3/envs/bittensor/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
def draw_pareto_graph(log_directory):
    accuracy_values = []
    num_parameters_values = []
    file_names = []
    assigned_users = []

    for filename in os.listdir(log_directory):
        if filename.endswith('.csv'):
            filepath = os.path.join(log_directory, filename)
            df = pd.read_csv(filepath, skiprows=lambda x: x % 3 != 0)
            if not df.empty:
                for index, row in df.iterrows():
                    response_list = eval(row['Response'])
                    accuracy_values.append(response_list[0])
                    num_parameters_values.append(response_list[1])
                    file_names.append(filename)
                    assigned_users.append(row.get('Assigned_User', 'N/A'))

    # Check if the lists are empty
    if not num_parameters_values or not accuracy_values:
        # Create an empty plot with a consistent style
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=[], y=[], mode='markers'))
        fig.update_layout(
            title='No data available',
            xaxis=dict(
                title='Number of Parameters',
                title_font=dict(color='#FFFFFF'),
                tickfont=dict(color='#FFFFFF'),
                linecolor='#FFFFFF',
                showgrid=False,
                range=[0, 1]
            ),
            yaxis=dict(
                title='Accuracy',
                title_font=dict(color='#FFFFFF'),
                tickfont=dict(color='#FFFFFF'),
                linecolor='#FFFFFF',
                showgrid=False,
                range=[0, 1]
            ),
            plot_bgcolor='#171B23',
            paper_bgcolor='#171B23',
            legend=dict(font=dict(color='#FFFFFF')),
            hovermode='closest',
            hoverlabel=dict(font=dict(color='#FFFFFF')),
            showlegend=False
        )
        return pio.to_html(fig, full_html=False)

    # Compute unique colors for each file
    unique_colors = [hashlib.sha256(name.encode()).hexdigest()[:6] for name in set(file_names)]

    scatter_traces = []
    for filename, color in zip(set(file_names), unique_colors):
        x_values = [num_parameters_values[i] for i, name in enumerate(file_names) if name == filename]
        y_values = [accuracy_values[i] for i, name in enumerate(file_names) if name == filename]
        hover_text = [f"uid:{assigned_users[i]}" for i, name in enumerate(file_names) if name == filename]
        scatter_trace = go.Scatter(
            x=x_values,
            y=y_values,
            mode='markers',
            name=filename.split('.')[0],
            marker=dict(color=f'#{color}'),
            hoverinfo='text',
            text=hover_text
        )
        scatter_traces.append(scatter_trace)

    pareto_optimal_points = find_pareto_optimal_points(accuracy_values, num_parameters_values)
    print(pareto_optimal_points)
    padding_factor = 0.01  # 10% padding
    x_min, x_max = min(num_parameters_values), max(num_parameters_values)
    y_min, y_max = min(accuracy_values), max(accuracy_values)
    x_range = [(1 - padding_factor) * x_min, (1 + padding_factor) * x_max]
    y_range = [(1 - padding_factor) * y_min, (1 + padding_factor) * y_max]

    fig = go.Figure(data=scatter_traces)
    if pareto_optimal_points:
        pareto_x, pareto_y = zip(*pareto_optimal_points)
        pareto_trace = go.Scatter(
            x=pareto_x,
            y=pareto_y,
            mode='lines',
            name='Pareto Optimal Line',
            line=dict(color='#ff0000')
        )
        fig.add_trace(pareto_trace)

    fig.update_layout(
        title='Scatter Plot of Multi-Objective Optimization Scores',
        xaxis=dict(
            title='Number of Parameters',
            range=x_range,
            title_font=dict(color='#FFFFFF'),
            tickfont=dict(color='#FFFFFF'),
            linecolor='#FFFFFF',
            showgrid=False
        ),
        yaxis=dict(
            title='Accuracy',
            range=y_range,
            title_font=dict(color='#FFFFFF'),
            tickfont=dict(color='#FFFFFF'),
            linecolor='#FFFFFF',
            showgrid=False
        ),
        plot_bgcolor='#171B23',
        paper_bgcolor='#171B23',
        legend=dict(font=dict(color='#FFFFFF')),
        hovermode='closest',
        hoverlabel=dict(font=dict(color='#FFFFFF')),
        showlegend=False
    )

    return fig

def find_pareto_optimal_points(accuracy, parameters):
    pareto_optimal = []
    for i in range(len(accuracy)):
        is_pareto = True
        for j in range(len(accuracy)):
            if accuracy[i] <= accuracy[j] and parameters[i] >= parameters[j] and i != j:
                is_pareto = False
                break
        if is_pareto:
            pareto_optimal.append((parameters[i], accuracy[i]))
    pareto_optimal.sort()
    return pareto_optimal

In [20]:
import pandas as pd
import os
import hashlib
import plotly.graph_objs as go
import plotly.io as pio

def print_pareto(log_directory):
    accuracy_values = []
    num_parameters_values = []
    file_names = []
    assigned_users = []
    genome_values = []  # Store Genome values

    for filename in os.listdir(log_directory):
        if filename.endswith('.csv'):
            filepath = os.path.join(log_directory, filename)
            df = pd.read_csv(filepath, skiprows=lambda x: x % 3 != 0)
            for index, row in df.iterrows():
                response_list = eval(row['Response'])
                accuracy_values.append(response_list[0])
                num_parameters_values.append(response_list[1])
                file_names.append(filename)
                assigned_users.append(row.get('Assigned_User', 'N/A'))
                genome_values.append(row['Genome'])  # Collect Genome values

    # Identify Pareto optimal points
    pareto_indices = find_pareto(accuracy_values, num_parameters_values)

    return pareto_indices, genome_values, assigned_users

def find_pareto(accuracy, parameters):
    pareto_optimal_indices = []
    for i in range(len(accuracy)):
        is_pareto = True
        for j in range(len(accuracy)):
            if accuracy[i] <= accuracy[j] and parameters[i] >= parameters[j] and i != j:
                is_pareto = False
                break
        if is_pareto:
            pareto_optimal_indices.append(i)
    return pareto_optimal_indices

pareto_indices, genome_values, assigned_users = print_pareto(nas_log_dir)
# This is selcted from the Pareto Optimal Line from the figure below
which_genome = 4
for i in range(len(pareto_indices)):
    if assigned_users[pareto_indices[i]] == which_genome:
            genome_string_str = genome_values[pareto_indices[i]]
            genome_string = ast.literal_eval(genome_string_str)
            print(genome_string)
    



[[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]


In [18]:
genome_string = [[0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [21]:
start_time = datetime.now()
performance = train_search.main(genome=genome_string,
                                            search_space = 'macro',
                                            init_channels = 36,
                                            layers=15, cutout=True,
                                            epochs=20,
                                            save='arch_{}'.format(1),
                                            expr_root='')
# Get the current time after running the code
end_time = datetime.now()

# Calculate the duration it took to run the code
duration = end_time - start_time
print(f"The code took {duration} to run.")


Experiment dir : arch_1
05/11 11:08:22 PM Architecture = [[[0], [1, 0], [0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 1, 1], [0, 1, 1, 0, 1, 0], [1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 1, 0, 1, 1, 0, 1, 0], [0]], [[0], [1, 1], [0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [1]], [[0], [0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [1]]]
05/11 11:08:22 PM param size = 0.399934MB
2024-05-11 23:08:23.182 |       INFO       | 👷 epoch 0 lr 0.02469315587086158


/home/nima/miniconda3/envs/bittensor/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/home/nima/miniconda3/envs/bittensor/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:854: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


2024-05-11 23:08:36.362 |       INFO       | 👷 train_acc 37.265            
2024-05-11 23:08:36.363 |       INFO       | 👷 epoch 1 lr 0.02393874741347802
2024-05-11 23:08:49.492 |       INFO       | 👷 train_acc 53.7025           
2024-05-11 23:08:49.492 |       INFO       | 👷 epoch 2 lr 0.02291005952837051
2024-05-11 23:09:02.623 |       INFO       | 👷 train_acc 59.895            
2024-05-11 23:09:02.623 |       INFO       | 👷 epoch 3 lr 0.021632279186225744
2024-05-11 23:09:15.871 |       INFO       | 👷 train_acc 63.5725           
2024-05-11 23:09:15.872 |       INFO       | 👷 epoch 4 lr 0.02013672046361853
2024-05-11 23:09:29.111 |       INFO       | 👷 train_acc 66.8375           
2024-05-11 23:09:29.112 |       INFO       | 👷 epoch 5 lr 0.018460049154373746
2024-05-11 23:09:42.442 |       INFO       | 👷 train_acc 69.2775           
2024-05-11 23:09:42.442 |       INFO       | 👷 epoch 6 lr 0.016643374554906734
2024-05-11 23:09:55.736 |       INFO       | 👷 train_acc 71.1825         

In [26]:
import plotly.graph_objs as go

data = results

# Choose the parameter to use for sorting (params or flops)
sort_key = 'flops'

# Sort the data based on the chosen parameter
sorted_data = sorted(data, key=lambda x: x[sort_key])

# Extract x and y values
x_values = [entry[sort_key] for entry in sorted_data]
y_values = [entry['valid_acc'] for entry in sorted_data]
uid_values = [entry['uid'] for entry in sorted_data]

# Create the scatter plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='markers', text=uid_values,
                          marker=dict(size=10),
                          hoverinfo='x+y+text', name='Validation Accuracy'))

# Add axis labels and title
if sort_key == 'params':
    fig.update_layout(xaxis_title='Parameters', yaxis_title='Validation Accuracy', title='Validation Accuracy vs Parameters')
else:
    fig.update_layout(xaxis_title='FLOPS', yaxis_title='Validation Accuracy', title='Validation Accuracy vs FLOPS')

# Set dark theme
fig.update_layout(template='plotly_dark')

# Show plot
fig.show()

: 